# **Turning Autograd off and on**

## requires_grad flag on tensor directly...

In [1]:
import torch

In [2]:
a = torch.ones(2,3,requires_grad=True)
print(a)

b1 = 2 * a
print(b1)

a.requires_grad=False
b2 = a * 2
print(b2)

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
tensor([[2., 2., 2.],
        [2., 2., 2.]], grad_fn=<MulBackward0>)
tensor([[2., 2., 2.],
        [2., 2., 2.]])


the value of "a" and "b1" will be require_grad ... but the "b2" hai no require_grad..

# Autograd turned_off temporarily...

In [3]:
# use torch.no_grad()
a = torch.ones(2,3,requires_grad=True)*2
b = torch.ones(2,3,requires_grad=True)*3

c1 = a + b
print(c1)

with torch.no_grad():
    c2 = a + b

print(c2)
c3 = a * b
print(c3)


tensor([[5., 5., 5.],
        [5., 5., 5.]], grad_fn=<AddBackward0>)
tensor([[5., 5., 5.],
        [5., 5., 5.]])
tensor([[6., 6., 6.],
        [6., 6., 6.]], grad_fn=<MulBackward0>)


In [4]:
def add_tensors1(x,y):
    return x+y

@torch.no_grad()
def add_tensors2(x,y):
    return x + y

a = torch.ones(2,3 , requires_grad=True)*2
b = torch.ones(2,3,requires_grad=True)*3


c1 = add_tensors1(a,b)
print(c1)
c2 = add_tensors2(a,b)
print(c2)

tensor([[5., 5., 5.],
        [5., 5., 5.]], grad_fn=<AddBackward0>)
tensor([[5., 5., 5.],
        [5., 5., 5.]])


# detach()

In [7]:
x = torch.rand(5 , requires_grad = True)
y = x.detach()

print(x)
print(y)

tensor([0.0352, 0.0162, 0.4228, 0.4547, 0.9030], requires_grad=True)
tensor([0.0352, 0.0162, 0.4228, 0.4547, 0.9030])


# Autograd Profiler

*   Autograd tracks every step of your computation in detail.
*   computation history combined with time information...



In [16]:
device = torch.device('cpu')
run_on_gpu = False
if torch.cuda.is_available():
    device = torch.device('cuda')
    run_on_gpu = True
x = torch.randn(2,3,requires_grad=True)
y = torch.rand(2,3,requires_grad=True)
z = torch.ones(2,3,requires_grad=True)

with torch.autograd.profiler.profile(use_cuda= run_on_gpu) as prf:
    for _ in range(1000):
        z = (z / x)*y

print(prf.key_averages().table(sort_by = "self_cpu_time_total"))

---------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
           Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
      aten::div        48.24%      13.483ms        50.65%      14.157ms      14.157us      14.032ms        50.49%      14.032ms      14.032us          1000  
      aten::mul        47.06%      13.153ms        49.35%      13.791ms      13.791us      13.761ms        49.51%      13.761ms      13.761us          1000  
    aten::empty         4.69%       1.311ms         4.69%       1.311ms       0.656us       0.000us         0.00%       0.000us       0.000us          2000  
---------------  ------------  ------------  ------